# Trabalho - Redes Neurais Artificiais

## Método adotado: Backpropagation

### Rafael Maia e Luís 

## Importação das bibliotecas necessárias

In [1]:
import matplotlib.pyplot as plt
from keras.datasets import mnist
import numpy as np
import operator

## Criação do modelo

In [2]:
class BP_Model:
    maxAcuracia = 0 
    def __init__(self, tamEntrada, tamCamadaEscondida, tamSaida, rate=0.1):
        self.dEntrada = tamEntrada
        self.dCamadaEscondida = tamCamadaEscondida
        self.dSaida = tamSaida
        self.wInH = np.random.rand(self.dEntrada, self.dCamadaEscondida)*0.2-0.1
        self.wOutH = np.random.rand(self.dCamadaEscondida, self.dSaida)*0.2-0.1
        self.delta_InH = np.zeros(self.dCamadaEscondida)
        self.delta_OutH = np.zeros(self.dSaida)
        self.learning_rate = rate
        self.custo = 0

    def forward(self, sample):
        ativCamadaOculta = self.tanh(np.dot(sample, self.wInH))
        activCamadaSaida = self.sigmoid(np.dot(ativCamadaOculta, self.wOutH))
        return ativCamadaOculta, activCamadaSaida

    def auxAcao(self, x, y):
        saida = self.forward(x)[1]
        return np.sum((np.array(saida) - np.array(y))**2)*0.5, np.array(saida)-np.array(y)

    def Acao(self, x, y):
        val = 0.
        vec = np.zeros(self.dSaida)
        for i in range(len(x)):
            temp = self.auxAcao(x[i],y[i])
            val += temp[0]
            vec += temp[1]
        return val, vec

    def auxBP(self, x, y):
        custo, custo_vec = self.auxAcao(x, y)
        activHidden, activSaida = self.forward(x)
        deltaSaida = custo_vec * activSaida * (1-activSaida)
        deltaH = np.dot(self.wOutH, deltaSaida) * (1-activHidden**2)
        self.wOutH -= self.learning_rate*deltaSaida*activHidden[:,np.newaxis]
        self.wInH -= self.learning_rate * deltaH * np.array(x)[:,np.newaxis]
        return self.wOutH, self.wInH, custo

    def BP(self, x, y):
        self.custo = 0
        tam = np.arange(len(x))
        np.random.shuffle(tam)
        for i in tam[:100]:
            wOutH, wInH, cost = self.auxBP(x[i], y[i])
            self.custo += cost
        self.custo /= len(x)
        return self.custo

    def Treinamento(self, x, y, x_Teste, y_Teste, epoca=100):
        for i in range(epoca):
            self.Teste(x_Teste, y_Teste)
            self.BP(x, y)
            print('Epoca: {}'.format(i+1))
            print('Custo: {}'.format(self.custo))
            print("/-----------------------------------------------/")


    def Teste(self, x, y):
        acuracia = 0
        for i in range(len(x)):
            saida = self.forward(x[i])[1]
            index, _ = max(enumerate(saida), key = operator.itemgetter(1))
            if y[i][index] == 1:
                acuracia += 1
            if acuracia / len(x) > self.maxAcuracia:
                self.maxAcuracia = acuracia / len(x)
        print('Acuracia: {}'.format(acuracia / len(x)))
        
    def MaxAcuracia(self):
         print('Acuracia maxima: {}'.format(self.maxAcuracia))


    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))

    @staticmethod
    def tanh(x):
        return (1-np.exp(-2*x))/(1+np.exp(-2*x))

## Obtenção dos dados

In [3]:
(x_Treinamento, y_Treinamento), (x_Teste, y_Teste) = mnist.load_data()

## Processamento

In [4]:
x_Treinamento = x_Treinamento.reshape(60000, 784)
x_Teste = x_Teste.reshape(10000, 784)

numClasses = 10

x_Treinamento = x_Treinamento.astype('float32')
x_Teste = x_Teste.astype('float32')
x_Treinamento /= 255
x_Teste /= 255

t_Treinamento = np.zeros((y_Treinamento.shape[0], numClasses))
t_Treinamento[np.arange(y_Treinamento.shape[0]), y_Treinamento] = 1
y_Treinamento = t_Treinamento

t_Teste = np.zeros((y_Teste.shape[0], numClasses))
t_Teste[np.arange(y_Teste.shape[0]), y_Teste] = 1
y_Teste = t_Teste

## Definição da camada escondida
O número de neurônios em cada camada é uma questão mais empírica, não existindo regras explícitas para um cálculo ideal. Jeff Heaton, o autor de <b><i>Introduction to Neural Networks for Java</i></b>, sugere três abordagens iniciais, que vamos aplicar como exemplo para uma rede contendo 30 neurônios na camada de entrada e 2 neurônios na camada de saída:

* O número de neurônios escondidos deve estar entre o tamanho da camada de entrada e o da camada de saída. Usar o número médio entre as duas camadas é uma boa opção; ou seja, no nosso exemplo, o valor de (30+2)/2 = 16 neurônios.
* O número de neurônios escondidos deve ser 2/3 do tamanho da camada de entrada, mais o tamanho da camada de saída. Assim, a camada escondida no nosso exemplo deve conter 30*2/3+2 = 22 neurônios.
* O número de neurônios escondidos deve ser menor que o dobro do tamanho da camada de entrada. Ou seja, no nosso exemplo, a camada escondida deve conter menos que 60 neurônios.

In [7]:
N_HIDEN_LAYER = 397

## Exibição dos resultados

In [8]:
BP_Model = BP_Model(784, N_HIDEN_LAYER, numClasses)
BP_Model.Treinamento(x_Treinamento, y_Treinamento, x_Teste, y_Teste)
BP_Model.MaxAcuracia()

TypeError: 'BP_Model' object is not callable